<a href="https://colab.research.google.com/github/unyime1/medical-cost-prediction/blob/main/normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [2]:
# read in insurance dataset
insurance_data = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [3]:
# view insurance data
insurance_data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [23]:
# normalize numerical features, and one-hot encode text based features

# Create a column transformaer
ct = make_column_transformer(
    # standardize values in these columns to 0 and 1
    (MinMaxScaler(), ["age", "bmi", "children"]), 
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"])
)

# create our feature(X) and labels(y)
X = insurance_data.drop("charges", axis=1)
y = insurance_data["charges"]

# create test splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Fit the column transformer to the training data
ct.fit(X_train)

# transform training and test data with normalization and one hot encoding
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [9]:
# view normalized data
X_train_normal[0]

array([0.60869565, 0.10734463, 0.4       , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        ])

In [24]:
# build a neural network to train on normalized data

# set seed
tf.random.set_seed(42)

# create a model with two dense layer
model = tf.keras.Sequential([
      tf.keras.layers.Dense(10),
      tf.keras.layers.Dense(1)
])

# compile the model
model.compile(
    loss=tf.keras.losses.mae,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    metrics=["mae"]
)

# fit the model
history = model.fit(X_train_normal, y_train, epochs=200)

Epoch 1/200
34/34 [==============================] - 0s 2ms/step - loss: 13128.3428 - mae: 13128.3428
Epoch 2/200
34/34 [==============================] - 0s 1ms/step - loss: 11425.7715 - mae: 11425.7715
Epoch 3/200
34/34 [==============================] - 0s 2ms/step - loss: 8831.5859 - mae: 8831.5859
Epoch 4/200
34/34 [==============================] - 0s 2ms/step - loss: 7828.7920 - mae: 7828.7920
Epoch 5/200
34/34 [==============================] - 0s 2ms/step - loss: 7545.1836 - mae: 7545.1836
Epoch 6/200
34/34 [==============================] - 0s 2ms/step - loss: 7226.4653 - mae: 7226.4653
Epoch 7/200
34/34 [==============================] - 0s 2ms/step - loss: 6853.6157 - mae: 6853.6157
Epoch 8/200
34/34 [==============================] - 0s 1ms/step - loss: 6406.4692 - mae: 6406.4692
Epoch 9/200
34/34 [==============================] - 0s 2ms/step - loss: 5860.9336 - mae: 5860.9336
Epoch 10/200
34/34 [==============================] - 0s 2ms/step - loss: 5227.5352 - mae: 5227.

In [25]:
# evaluate model
model.evaluate(X_test_normal, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3168.1599 - mae: 3168.1599


[3168.159912109375, 3168.159912109375]